In [3]:
import pandas as pd
import json
import ast
import re
import datasets
import transformers
from accelerate import disk_offload
from transformers import (
    AutoModel,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    BitsAndBytesConfig)
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
from sklearn.metrics import accuracy_score, f1_score, classification_report
from datasets import load_dataset, load_from_disk, Dataset, DatasetDict


# Load Data

In [4]:
df_train = pd.read_csv('/projects/melodi/mettaleb/Textmine//data/train.csv')
#df_train = df_train.set_index("id")
df_train.entities = df_train.entities.apply(json.loads)
df_train.relations = df_train.relations.apply(json.loads)
df_train.head()

,id,text,entities,relations
0,181,"Anam Destresse, président de l'ONG ""Ma passion...","[{'id': 0, 'mentions': [{'value': 'accident', ...","[[0, STARTED_IN, 9], [7, IS_LOCATED_IN, 9], [5..."
1,31669,"À Paris, le 8 avril 2022, l'usine de déodorant...","[{'id': 0, 'mentions': [{'value': 'explosé', '...","[[9, IS_LOCATED_IN, 8], [11, OPERATES_IN, 8], ..."
2,51470,"En Espagne, dans une région agricole, une cont...","[{'id': 0, 'mentions': [{'value': 'contaminati...","[[7, IS_PART_OF, 8], [9, OPERATES_IN, 1], [0, ..."
3,51332,Un important incendie a fait des ravages dans ...,"[{'id': 0, 'mentions': [{'value': 'incendie', ...","[[12, IS_IN_CONTACT_WITH, 5], [0, IS_LOCATED_I..."
4,1131,« Je coule » : onze heures après avoir envoyé ...,"[{'id': 0, 'mentions': [{'value': 'renversé', ...","[[9, IS_LOCATED_IN, 2], [0, START_DATE, 17], [..."


In [8]:
df_train.iloc[600]["text"]

'En Europe, la crise financière qui frappait sévèrement le secteur bancaire depuis plusieurs mois s’est généralisée pour devenir une crise économique. Cette crise a affecté les transports, l’énergie et le domaine de la santé. Pour arrêter la saignée, certains gouvernements, comme le gouvernement portugais, ont décidé d’effectuer une découpe dans certaines dépenses publiques. En matière de santé par exemple, certains médicaments ont cessé d’être pris en charge. Dans le domaine des transports, le prix des billets de train a fortement augmenté. Malheureusement, comme l’avait prédit le sociologue Bruno Alves dans une revue, ces mesures drastiques ont entraîné une forte tension sociale ainsi que la montée de l’insécurité. Certains quartiers précaires à Lisbonne sont devenus de véritables coupe-gorges et marchés noirs où des articles volés comme des pièces détachées de voitures ou des denrées alimentaires sont vendus.\n'

In [7]:
df_train.iloc[600]["relations"]

[[12, 'IS_LOCATED_IN', 4],
 [3, 'STARTED_IN', 11],
 [0, 'IS_LOCATED_IN', 4],
 [7, 'HAS_QUANTITY', 22],
 [0, 'HAS_CONSEQUENCE', 3],
 [6, 'OPERATES_IN', 11],
 [6, 'IS_OF_NATIONALITY', 20],
 [1, 'IS_LOCATED_IN', 4],
 [3, 'IS_LOCATED_IN', 12],
 [2, 'IS_LOCATED_IN', 4],
 [2, 'HAS_CONSEQUENCE', 3],
 [9, 'IS_LOCATED_IN', 4],
 [1, 'HAS_CONSEQUENCE', 3],
 [2, 'STARTED_IN', 11],
 [0, 'IS_LOCATED_IN', 12],
 [0, 'HAS_CONSEQUENCE', 2],
 [9, 'GENDER_MALE', 9],
 [1, 'IS_LOCATED_IN', 12],
 [3, 'IS_LOCATED_IN', 11],
 [2, 'IS_LOCATED_IN', 12],
 [0, 'STARTED_IN', 4],
 [0, 'START_DATE', 21],
 [1, 'HAS_CONSEQUENCE', 2],
 [9, 'HAS_CATEGORY', 17],
 [0, 'IS_LOCATED_IN', 11],
 [0, 'HAS_CONSEQUENCE', 1],
 [11, 'IS_LOCATED_IN', 12],
 [3, 'IS_LOCATED_IN', 4],
 [6, 'OPERATES_IN', 12],
 [1, 'IS_LOCATED_IN', 11],
 [5, 'OPERATES_IN', 4],
 [9, 'HAS_CONTROL_OVER', 10],
 [2, 'IS_LOCATED_IN', 11]]

In [6]:
df_train.iloc[600]["entities"]

[{'id': 0,
  'mentions': [{'value': 'crise financière', 'start': 14, 'end': 30}],
  'type': 'ECONOMICAL_CRISIS'},
 {'id': 1,
  'mentions': [{'value': 'crise économique', 'start': 132, 'end': 148},
   {'value': 'crise', 'start': 156, 'end': 161},
   {'value': 'augmenté', 'start': 537, 'end': 545}],
  'type': 'ECONOMICAL_CRISIS'},
 {'id': 2,
  'mentions': [{'value': 'volés', 'start': 839, 'end': 844}],
  'type': 'THEFT'},
 {'id': 3,
  'mentions': [{'value': 'vendus', 'start': 917, 'end': 923}],
  'type': 'TRAFFICKING'},
 {'id': 4,
  'mentions': [{'value': 'Europe', 'start': 3, 'end': 9}],
  'type': 'PLACE'},
 {'id': 5,
  'mentions': [{'value': 'gouvernements', 'start': 259, 'end': 272}],
  'type': 'NON_MILITARY_GOVERNMENT_ORGANISATION'},
 {'id': 6,
  'mentions': [{'value': 'gouvernement', 'start': 283, 'end': 295}],
  'type': 'NON_MILITARY_GOVERNMENT_ORGANISATION'},
 {'id': 7,
  'mentions': [{'value': 'médicaments', 'start': 419, 'end': 430}],
  'type': 'MATERIEL'},
 {'id': 8,
  'mention

In [5]:
df_test = pd.read_csv('/projects/melodi/mettaleb/Textmine/data/test.csv')
#df_train = df_train.set_index("id")
df_test.entities = df_test.entities.apply(json.loads)
#df_test.relations = df_test.relations.apply(json.loads)
df_test.head()

,id,text,entities
0,1204,“FEAR” est une organisation spécialisée dans l...,"[{'id': 0, 'mentions': [{'value': 'crash', 'st..."
1,4909,"48 heures après le braquage du ""DC Supermarket...","[{'id': 0, 'mentions': [{'value': 'DC Supermar..."
2,2353,"À Genève, la boîte de nuit ""Pretty Woman"" a en...","[{'id': 0, 'mentions': [{'value': 'tombé', 'st..."
3,1210,"À Caracas, l'an dernier, une mine située à l'o...","[{'id': 0, 'mentions': [{'value': 'effondremen..."
4,41948,"À Genève, une industrie de fabrication d'arbre...","[{'id': 0, 'mentions': [{'value': 'incendiée',..."


In [6]:
df_test[df_test["id"] == 1204]["text"].values[0]

"“FEAR” est une organisation spécialisée dans le transport des tablettes et des smartphones basée à Buenos Aires, d'une latitude de -34.6083. Il s'agit de la plus grande métropole cosmopolite de l'Argentine. Il a été rapporté que le dernier avion commercial de “FEAR” a fait un crash aérien en novembre dernier. Cela est dû à une perte de contrôle de l'appareil. Au bord du vol se trouvaient les pilotes M. Clavier Joshua et M. Joseph Charles. Tous deux ont perdu la vie dans ce terrible accident. Le serveur informatique et le signal émis par la satellite ont permis aux secours de localiser l'appareil."

In [7]:
df_test[df_test['id']==1204]["entities"][0]

[{'id': 0,
  'mentions': [{'value': 'crash', 'start': 277, 'end': 282},
   {'value': 'accident', 'start': 487, 'end': 495}],
  'type': 'ACCIDENT'},
 {'id': 1,
  'mentions': [{'value': 'Clavier Joshua', 'start': 406, 'end': 420}],
  'type': 'CIVILIAN'},
 {'id': 2,
  'mentions': [{'value': 'Joseph Charles', 'start': 427, 'end': 441}],
  'type': 'CIVILIAN'},
 {'id': 3,
  'mentions': [{'value': 'appareil', 'start': 352, 'end': 360},
   {'value': 'appareil', 'start': 594, 'end': 602},
   {'value': 'avion commercial', 'start': 240, 'end': 256}],
  'type': 'MATERIEL'},
 {'id': 4,
  'mentions': [{'value': 'serveur informatique', 'start': 500, 'end': 520}],
  'type': 'MATERIEL'},
 {'id': 5,
  'mentions': [{'value': 'satellite', 'start': 546, 'end': 555}],
  'type': 'MATERIEL'},
 {'id': 6,
  'mentions': [{'value': 'FEAR', 'start': 261, 'end': 265},
   {'value': 'FEAR', 'start': 1, 'end': 5}],
  'type': 'NON_GOVERNMENTAL_ORGANISATION'},
 {'id': 7,
  'mentions': [{'value': 'smartphones', 'start': 

In [8]:
relations = ['RESIDES_IN', 'HAS_CONTROL_OVER', 'IS_COOPERATING_WITH', 'IS_BORN_ON', 'HAS_LONGITUDE', 'HAS_LATITUDE', 'IS_OF_SIZE', 'IS_BORN_IN', 'IS_PART_OF', 'IS_AT_ODDS_WITH', 'IS_LOCATED_IN', 'STARTED_IN', 'IS_OF_NATIONALITY', 'HAS_COLOR', 'WEIGHS', 'IS_REGISTERED_AS', 'START_DATE', 'INJURED_NUMBER', 'DIED_IN', 'DEATHS_NUMBER', 'CREATED', 'GENDER_FEMALE', 'INITIATED', 'HAS_FOR_HEIGHT', 'HAS_FOR_LENGTH', 'HAS_CATEGORY', 'HAS_CONSEQUENCE', 'WAS_CREATED_IN', 'HAS_FAMILY_RELATIONSHIP', 'OPERATES_IN', 'WAS_DISSOLVED_IN', 'IS_DEAD_ON', 'HAS_QUANTITY', 'HAS_FOR_WIDTH', 'END_DATE', 'IS_IN_CONTACT_WITH', 'GENDER_MALE']
relations_trad = {'RESIDES_IN': 'RÉSIDE_DANS','IS_OF_SIZE': 'EST_DE_TAILLE',    'IS_BORN_ON': 'EST_NÉ_LE',    'CREATED': 'CRÉÉ',    'HAS_CONSEQUENCE': 'A_DES_CONSÉQUENCES',    'HAS_FOR_LENGTH': 'A_POUR_LONGUEUR',    'DIED_IN': 'MORT_EN',    'START_DATE': 'DATE_DE_DÉBUT',    'INITIATED': 'INITIÉ',    'HAS_CATEGORY': 'A_UNE_CATEGORIE',    'HAS_LATITUDE': 'A_LA_LATITUDE',    'GENDER_FEMALE': 'GENRE_FEMININ',    'DEATHS_NUMBER': 'NOMBRE_DE_MORTS',    'GENDER_MALE': 'GENRE_MASCULIN',    'IS_PART_OF': 'FAIT_PARTIE_DE',    'WEIGHS': 'PESENT',    'IS_REGISTERED_AS': 'EST_ENREGISTRÉ_COMME',    'HAS_QUANTITY': 'A_UNE_QUANTITÉ',    'IS_OF_NATIONALITY': 'EST_DE_NATIONALITÉ',    'INJURED_NUMBER': 'NOMBRE_DE_BLESSÉS',    'END_DATE': 'DATE_DE_FIN',    'HAS_CONTROL_OVER': 'A_UNE_CE_CONTROLE_SUR',    'IS_COOPERATING_WITH': 'COOPÈRE_AVEC',    'IS_BORN_IN': 'EST_NÉ_A',    'HAS_FOR_WIDTH': 'A_POUR_LARGEUR',    'IS_AT_ODDS_WITH': 'EST_EN_CONFLIT_AVEC',    'HAS_COLOR': 'A_COULEUR',    'HAS_FAMILY_RELATIONSHIP': 'A_UNE_RELATION_FAMILIALE',    'WAS_DISSOLVED_IN': 'A_ÉTÉ_DISSOUTE_EN',    'HAS_FOR_HEIGHT': 'A_POUR_HAUTEUR',    'IS_DEAD_ON': 'EST_MORT_LE',    'STARTED_IN': 'A_COMMENCÉ_EN',    'OPERATES_IN': 'OPÈRE_EN',    'IS_LOCATED_IN': 'EST_LOCALISÉ_EN',    'WAS_CREATED_IN': 'A_ÉTÉ_CRÉÉ_EN',    'HAS_LONGITUDE': 'A_LA_LONGITUDE','IS_IN_CONTACT_WITH': 'EST_EN_CONTACT_AVEC', 'PAS_DE_RELATION': 'PAS_DE_RELATION'}
dif_entities = {
    "Actor": ["Organization",  "Organisation", "Person"],
    "Organization": ["Gouvernement Organization"  , "Group of Individuals", "Intergovernmental Organization", "Intergovernmental Organisation", "Non Governmental Organization", "Non Governmental Organisation"],
    "Organisation": ["Gouvernement Organisation", "Group of Individuals", "Intergovernmental Organization", "Intergovernmental Organisation", "Non Governmental Organization", "Non Governmental Organisation"],
    "Gouvernement Organization": ["Military Organization", "Non Military Organization","NON_MILITARY_GOVERNMENT_ORGANIZATION"],
    "Gouvernement Organisation": ["Military Organisation", "Non Military Organisation","NON_MILITARY_GOVERNMENT_ORGANISATION"],
    "Person": ["Civilian", "Criminal", "Military"],
    "Event": ["Accident", "CBRN Event", "Civil Unrest", "Criminal Event", "Large Scale Event"],
    "Civil Unrest": ["Agitating Trouble Making", "Civil War Outbreak", "Coup d'État", "Demonstration", "Election",
                     "Gathering", "Illegal Civil Demonstration", "Natural Causes Death", "Riot", "Strike", "Suicide"],
    "Criminal Event": ["Bombing", "Criminal Arrest", "Drug Operation", "Hooliganism Trouble-making", "Political Violence", "Theft", "Trafficking"],
    "Large Scale Event": ["Economical Crisis", "Epidemic", "Fire", "Natural Event", "Pollution"],
    "Material": ["MATERIEL","Material"],
    "Place": ["Place"],
    "TERRORIST_OR_CRIMINAL": ['TERRORIST_OR_CRIMINAL']
}
dif_Attrinuts = {"Category": 0,"Color": 0,"First Name": 0,"Height": 0,"Last Name": 0,"Length": 0,"Material Reference": 0,"Nationality": 0,"Quantity": ["Quantity Exact","Quantity Fuzzy","Quantity Max","Quantity Min"],"Time": ["Time Exact","Time Fuzzy","Time Max","Time Min"]}
Relations_Definition = ['Actor,Is_Located_In,Place','Event,Is_Located_In,Place','Place,Is_Located_In,Place','Actor,Is_of_Nationality,Nationality','Place,Is_of_Nationality,Nationality', 'Actor,Created,Organization','Actor,Has_Control_Over,Actor','Actor,Has_Control_Over,Material','Actor,Has_Control_Over,Place','Actor,Initiated,Event', 'Actor,Is_At_Odds_With,Actor','Actor,Is_Cooperating_With,Actor','Actor,Is_In_Contact_With,Actor','Actor,Is_Part_Of,Organization','Event,Deaths_Number,Quantity', 'Event,End_Date,Time','Event,Has_Consequence,Event','Event,Injured_Number,Quantity','Event,Start_Date,Time','Event,Started_In,Place', 'Material,Has_Color,Color','Material,Has_for_Height,Height','Material,Has_for_Length,Length','Material,Has_for_Width,Width','Material,Has_Quantity,Quantity', 'Material,Is_Registered_As,Material reference', 'Material,Weighs,Weight', 'Organization,Was_Created_In,Time', 'Organization,Was_Dissolved_In,Time', 'Organization,Is_Of_Size,Quantity', 'Organization,Operates_In,Place', 'Person,Died_In,Event', 'Person,Has_Category,Category', 'Person,Has_Family_Relationship,Person',  'Person,Gender_Female,N/A', 'Person,Gender_Male,N/A', 'Person,Is_Born_In,Place','Person,Is_Born_On,Time', 'Person,Is_Dead_On,Time', 'Person,Resides_In,Place', 'Place,Is_Located_In,Place','Actor,Is_of_Nationality,Nationality','Place,Is_of_Nationality,Nationality', 'Place,Has_Latitude,Latitude','Place,Has_Longitude,Longitude']

In [9]:
with open("/projects/melodi/mettaleb/Textmine/strategie5/soumission_strategie3.sent") as f :
    paragraphes_soumission = f.readlines()
with open("/projects/melodi/mettaleb/Textmine/strategie5/soumission_pairs.sent") as f :
    pairs_soumission = f.readlines()
with open("/projects/melodi/mettaleb/Textmine/strategie5/relation_possibles_strategie3_soumission.sent") as f :
    relation_possibles_strategie3_soumission = f.readlines()


In [10]:
import pandas as pd
fich_content = [x.strip() for x in pairs_soumission]
paragraphes_soumission = [x.strip() for x in paragraphes_soumission]
relation_possibles_strategie3_soumission = [x.strip() for x in relation_possibles_strategie3_soumission]

# Liste pour stocker les résultats
results = []

# Parcourir chaque ligne du fichier fich
for idx_row, line in enumerate(fich_content):
    # Extraction des informations de chaque ligne
    parts = line.split(" ; ")
    id_fich = int(parts[0].strip())
    e1_fich = parts[1].strip()
    e2_fich = parts[2].strip()
    
    # Chercher dans df la ligne correspondante
    df_row = df_test[df_test["id"] == id_fich]
    df_text = df_test[df_test["id"] == id_fich]["text"].values[0]
    if not df_row.empty:
        entities = df_row.iloc[0]["entities"]
        
        # Chercher les dictionnaires d'entités pour e1 et e2
        e1_dict = next((ent for ent in entities if any(mention['value'] == e1_fich for mention in ent['mentions'])), None)
        e2_dict = next((ent for ent in entities if any(mention['value'] == e2_fich for mention in ent['mentions'])), None)
        
        # Si les deux entités sont trouvées, les ajouter aux résultats
        if e1_dict and e2_dict:
            results.append({"id": id_fich, "entité1": e1_dict, "entité2": e2_dict, "relations_possibles": relation_possibles_strategie3_soumission[idx_row],"text": df_text})

# Créer un DataFrame avec les résultats
df_soumission5 = pd.DataFrame(results)


In [53]:
df_soumission5.head()

,id,entité1,entité2,relations_possibles,text,relations
0,1204,"{'id': 11, 'mentions': [{'value': 'Tous deux',...","{'id': 10, 'mentions': [{'value': 'Buenos Aire...",Is_Located_In;Has_Control_Over;PAS_DE_RELATION,“FEAR” est une organisation spécialisée dans l...,PAS_DE_RELATION
1,1204,"{'id': 2, 'mentions': [{'value': 'Joseph Charl...","{'id': 0, 'mentions': [{'value': 'crash', 'sta...",Died_In;PAS_DE_RELATION,“FEAR” est une organisation spécialisée dans l...,PAS_DE_RELATION
2,1204,"{'id': 2, 'mentions': [{'value': 'Joseph Charl...","{'id': 11, 'mentions': [{'value': 'Tous deux',...",Has_Control_Over;Is_At_Odds_With;Is_Cooperatin...,“FEAR” est une organisation spécialisée dans l...,PAS_DE_RELATION
3,1204,"{'id': 11, 'mentions': [{'value': 'Tous deux',...","{'id': 8, 'mentions': [{'value': 'tablettes', ...",Has_Control_Over;PAS_DE_RELATION,“FEAR” est une organisation spécialisée dans l...,PAS_DE_RELATION
4,1204,"{'id': 1, 'mentions': [{'value': 'Clavier Josh...","{'id': 7, 'mentions': [{'value': 'smartphones'...",Has_Control_Over;PAS_DE_RELATION,“FEAR” est une organisation spécialisée dans l...,PAS_DE_RELATION


In [49]:
df_soumission5.shape

(73692, 5)

In [50]:
relations_facke = ["PAS_DE_RELATION"]*73692

In [52]:
df_soumission5["relations"] = relations_facke

In [12]:
with open("/projects/melodi/mettaleb/Textmine/strategie5/test_strategie3.tup") as f :
    pairs_test = f.readlines()
with open("/projects/melodi/mettaleb/Textmine/strategie5/relation_possibles_strategie3_test.sent") as f :
    relation_possibles_strategie3_test = f.readlines()

fich_content = [x.strip() for x in pairs_test]
relation_possibles_strategie3_test = [x.strip() for x in relation_possibles_strategie3_test]
# Liste pour stocker les résultats
results = []

# Parcourir chaque ligne du fichier fich
for id_row, line in enumerate(fich_content):
    # Extraction des informations de chaque ligne
    parts = line.split(" ; ")
    id_fich = int(parts[0].strip())
    e1_fich = parts[1].strip()
    e2_fich = parts[2].strip()
    rel = parts[3].strip()
    
    # Chercher dans df la ligne correspondante
    df_row = df_train[df_train["id"] == id_fich]
    df_text = df_train[df_train["id"] == id_fich]["text"].values[0]
    if not df_row.empty:
        entities = df_row.iloc[0]["entities"]
        
        # Chercher les dictionnaires d'entités pour e1 et e2
        e1_dict = next((ent for ent in entities if any(mention['value'] == e1_fich for mention in ent['mentions'])), None)
        e2_dict = next((ent for ent in entities if any(mention['value'] == e2_fich for mention in ent['mentions'])), None)
        
        # Si les deux entités sont trouvées, les ajouter aux résultats
        if e1_dict and e2_dict:
            results.append({"id": id_fich, "entité1": e1_dict, "entité2": e2_dict, "relations": rel, "relations_possibles" : relation_possibles_strategie3_test[id_row], "text": df_text})

#Créer un DataFrame avec les résultats
df_test5 = pd.DataFrame(results)

In [13]:
with open("/projects/melodi/mettaleb/Textmine/strategie5/train_strategie3.tup") as f :
    pairs_train = f.readlines()
with open("/projects/melodi/mettaleb/Textmine/strategie5/relation_possibles_strategie3_train.sent") as f :
    relation_possibles_strategie3_train = f.readlines()

fich_content = [x.strip() for x in pairs_train]
relation_possibles_strategie3_train = [x.strip() for x in relation_possibles_strategie3_train]
# Liste pour stocker les résultats
results = []

# Parcourir chaque ligne du fichier fich
for id_row, line in enumerate(fich_content):
    # Extraction des informations de chaque ligne
    parts = line.split(" ; ")
    id_fich = int(parts[0].strip())
    e1_fich = parts[1].strip()
    e2_fich = parts[2].strip()
    rel = parts[3].strip()
    
    # Chercher dans df la ligne correspondante
    df_row = df_train[df_train["id"] == id_fich]
    df_text = df_train[df_train["id"] == id_fich]["text"].values[0]
    if not df_row.empty:
        entities = df_row.iloc[0]["entities"]
        
        # Chercher les dictionnaires d'entités pour e1 et e2
        e1_dict = next((ent for ent in entities if any(mention['value'] == e1_fich for mention in ent['mentions'])), None)
        e2_dict = next((ent for ent in entities if any(mention['value'] == e2_fich for mention in ent['mentions'])), None)
        
        # Si les deux entités sont trouvées, les ajouter aux résultats
        if e1_dict and e2_dict:
            results.append({"id": id_fich, "entité1": e1_dict, "entité2": e2_dict, "relations": rel, "relations_possibles": relation_possibles_strategie3_train[id_row], "text": df_text})

#Créer un DataFrame avec les résultats
df_train5 = pd.DataFrame(results)

In [14]:
df_train5.tail()

,id,entité1,entité2,relations,relations_possibles,text
31354,41535,"{'id': 0, 'mentions': [{'value': 'élections pr...","{'id': 1, 'mentions': [{'value': 'Portugal', '...",IS_LOCATED_IN,Is_Located_In;Started_In;PAS_DE_RELATION,"Au Portugal, le second tour des élections prés..."
31355,41535,"{'id': 8, 'mentions': [{'value': 'votants', 's...","{'id': 4, 'mentions': [{'value': 'bureaux de v...",IS_LOCATED_IN,Is_Located_In;Has_Control_Over;PAS_DE_RELATION,"Au Portugal, le second tour des élections prés..."
31356,41535,"{'id': 2, 'mentions': [{'value': 'forces de l’...","{'id': 4, 'mentions': [{'value': 'bureaux de v...",OPERATES_IN,Is_Located_In;Has_Control_Over;PAS_DE_RELATION,"Au Portugal, le second tour des élections prés..."
31357,41535,"{'id': 8, 'mentions': [{'value': 'votants', 's...","{'id': 7, 'mentions': [{'value': 'urne', 'star...",HAS_CONTROL_OVER,Has_Control_Over;PAS_DE_RELATION,"Au Portugal, le second tour des élections prés..."
31358,41535,"{'id': 9, 'mentions': [{'value': 'Auguste Ferr...","{'id': 9, 'mentions': [{'value': 'Auguste Ferr...",GENDER_MALE,Gender_Female;Gender_Male;PAS_DE_RELATION,"Au Portugal, le second tour des élections prés..."


In [20]:
df_test5.shape

(10299, 6)

### Equilibre Data

In [42]:
freq_reference_train = {'PAS_DE_RELATION': 1000, 'IS_LOCATED_IN': 700, 'IS_OF_SIZE': 300, 'HAS_CATEGORY': 400, 'STARTED_IN': 300, 'START_DATE': 200, 'GENDER_FEMALE': 300, 'OPERATES_IN': 300, 'END_DATE': 100, 'HAS_COLOR': 68, 'HAS_QUANTITY': 100, 'GENDER_MALE': 300, 'IS_PART_OF': 300, 'IS_IN_CONTACT_WITH': 300, 'CREATED': 90, 'INITIATED': 300, 'HAS_CONTROL_OVER': 300, 'HAS_FOR_LENGTH': 11, 'HAS_CONSEQUENCE': 300, 'IS_OF_NATIONALITY': 100, 'IS_AT_ODDS_WITH': 300, 'HAS_FOR_HEIGHT': 9, 'RESIDES_IN': 67, 'IS_BORN_IN': 21, 'IS_BORN_ON': 18, 'IS_COOPERATING_WITH': 300, 'INJURED_NUMBER': 57, 'HAS_FAMILY_RELATIONSHIP': 200, 'IS_DEAD_ON': 53, 'WAS_CREATED_IN': 100,  'IS_REGISTERED_AS': 26, 'DEATHS_NUMBER': 55, 'WAS_DISSOLVED_IN': 50 , 'DIED_IN': 20}
freq_reference_test  = {'PAS_DE_RELATION': 500, 'IS_LOCATED_IN': 300, 'IS_OF_SIZE': 100, 'HAS_CATEGORY': 100, 'STARTED_IN': 100, 'START_DATE': 100, 'GENDER_FEMALE': 100, 'OPERATES_IN': 100, 'END_DATE': 100, 'HAS_COLOR': 68, 'HAS_QUANTITY': 100, 'GENDER_MALE': 100, 'IS_PART_OF': 100, 'IS_IN_CONTACT_WITH': 100, 'CREATED': 90, 'INITIATED': 100, 'HAS_CONTROL_OVER': 100, 'HAS_FOR_LENGTH': 11, 'HAS_CONSEQUENCE': 100, 'IS_OF_NATIONALITY': 100, 'IS_AT_ODDS_WITH': 100, 'HAS_FOR_HEIGHT': 9, 'RESIDES_IN': 67, 'IS_BORN_IN': 21, 'IS_BORN_ON': 18, 'IS_COOPERATING_WITH': 100, 'INJURED_NUMBER': 57, 'HAS_FAMILY_RELATIONSHIP': 100, 'IS_DEAD_ON': 53, 'WAS_CREATED_IN': 50, 'IS_REGISTERED_AS': 26, 'DEATHS_NUMBER': 55, 'WAS_DISSOLVED_IN': 50 , 'DIED_IN': 20}

In [43]:
import pandas as pd


df = df_train5.sample(10299)

# Initialisation d'une liste pour les sous-échantillons
balanced_samples = []

# Itérer sur chaque relation et sous-échantillonner si nécessaire
for relation, max_count in freq_reference_train.items():
    # Filtrer le DataFrame pour cette relation
    relation_df = df[df['relations'] == relation]
    
    # Sous-échantillonnage si la fréquence actuelle dépasse la limite
    if len(relation_df) > max_count:
        relation_df = relation_df.sample(n=max_count, random_state=1)
    
    # Ajouter le sous-échantillon au résultat final
    balanced_samples.append(relation_df)

# Combiner tous les sous-échantillons en un DataFrame équilibré
df_balanced_train = pd.concat(balanced_samples, ignore_index=True)



In [44]:
df_balanced_train.shape , df_balanced_test.shape 

((5025, 6), (2392, 6))

### Prompt 

In [19]:
#with open('mydata/relations.txt') as f:
#relations = [r.strip() for r in set(df_train['relation'].to_list())]

    
def get_instruction(sent, tuples, rel_possibles, with_orig=True, with_cls=False):
    
    instructions, inputs, outputs = [], [], []
    if with_orig:
        instructions.append(f"Given a paragraph that describes the relationship between two words/phrases as options, extract the word/phrase pair and the corresponding lexical relationship between them from the input text. The output format should be \"{{relation1: word1, word2}}; {{relation2: word3, word4}}\". Options: {', '.join(relations)}. ")
        instructions.append(f"Given the input paragraph, please extract the subject and object containing a certain relation in the sentence according to the following relation types, in the format of \"{{relation1: word1, word2}}; {{relation2: word3, word4}}\". Relations include: {'; '.join(relations)}.")
        instructions.append(f"Provide a paragraph containing relationships between entities. Extract and identify the specific relations between entity pairs mentioned in the paragraph. Output the results in the format \"{{relation1: word1, word2}}; {{relation2: word3, word4}}\". Relations should be determined based on the context provided in the paragraph. Relations include: {'; '.join(relations)}.")

        inputs.extend([sent] * 3)
        outputs.extend(["; ".join([f"{tup[-1]}: {tup[0]}, {tup[1]}" for tup in tuples])] * 3)
    
    if with_cls:
        for tup in tuples:
            #instructions.append(f"Utilize the input text as a context reference, choose the right relationship between {tup[0]} and {tup[1]} from the options. Options: {', '.join(relations)}.")
            instructions.append(f"""Quelle est la relation entre les deux entités suivantes dans le contexte du texte d'entrée. Choisissez une réponse parmi : [ {rel_possibles} ].\n
            Entité 1 : {tup[0]}\n
            Entité 2 : {tup[1]}""")
            inputs.extend([sent] * 1)
            outputs.extend([tup[-1]] * 1)
    
    return instructions, inputs, outputs


def get_finred_dataset(df , with_orig, with_cls):

    instructions, inputs, outputs = [], [], []
    tuples_list_fr = []
    
    sentences = df['text'].to_list()
    #relations = df['relations_possibles'].to_list()
    relations = list(set(df['relations'].to_list()))

#############################################################################################################
    relations_fr = []
    for rel in relations :
        lis_rel = rel.split(";")
        list_rel_fr =[]
        for i in range(len(lis_rel)):
            rel = lis_rel[i].strip().upper()
            rel_fr = relations_trad[rel]
            list_rel_fr.append(rel_fr)
        rel_poss_fr = " ; ".join(list_rel_fr)
        relations_fr.append(rel_poss_fr)
    relations = relations_fr

###############################################################################################################
    
    for i in range(len(df['entité1'].to_list())):
        e1, e2, rel = df['entité1'].to_list()[i], df['entité2'].to_list()[i], df['relations'].to_list()[i]
        rel_fr = relations_trad[rel]
        tuples_list_fr.append([e1, e2,rel_fr])
    tuples_list = tuples_list_fr
###############################################################################################################

    for sent, tuples, rel_possibles in zip(sentences, tuples_list, relations):
        #tuples = [[e.strip() for e in tup.split(' ; ')] for tup in tuples]
        rel_possibles = " ; ".join(list(set(rel_possibles.split(" ; "))))
        ins, i, o = get_instruction(sent, tuples, rel_possibles, with_orig, with_cls)
        
        instructions.extend(ins)
        inputs.extend(i)
        outputs.extend(o)
    print("longuer = ",len(outputs))
    print("exemple : ",outputs[:10])
    print("exemple Instruciton: ",instructions[10])    
    return Dataset.from_dict({
        'input': inputs,
        'output': outputs,
        'instruction': instructions
    })
    

In [45]:
def get_instruction(sent, tuples, rel_possibles, with_orig=True, with_cls=False):
    instructions, inputs, outputs = [], [], []
    
    if with_orig:
        instructions.append(f"Given a paragraph that describes the relationship between two words/phrases as options, extract the word/phrase pair and the corresponding lexical relationship between them from the input text. The output format should be \"{{relation1: word1, word2}}; {{relation2: word3, word4}}\". Options: {', '.join(relations)}.")
        instructions.append(f"Given the input paragraph, please extract the subject and object containing a certain relation in the sentence according to the following relation types, in the format of \"{{relation1: word1, word2}}; {{relation2: word3, word4}}\". Relations include: {'; '.join(relations)}.")
        instructions.append(f"Provide a paragraph containing relationships between entities. Extract and identify the specific relations between entity pairs mentioned in the paragraph. Output the results in the format \"{{relation1: word1, word2}}; {{relation2: word3, word4}}\". Relations should be determined based on the context provided in the paragraph. Relations include: {'; '.join(relations)}.")

        inputs.extend([sent] * 3)
        outputs.extend(["; ".join([f"{tup[-1]}: {tup[0]}, {tup[1]}" for tup in tuples])] * 3)
    
    if with_cls:
        for tup in tuples:
            if len(tup) < 3:
                print("Tuple format incorrect, skipping:", tup)
                continue

            instructions.append(f"""D'après le contexte du texte d’entrée, identifiez la relation spécifique entre les deux entités suivantes. Choisissez une réponse parmi : [ {rel_possibles} ].\n
Entité 1 :\n
    Mention(s) Entité 1 : {tup[0]['mentions']}\n
    Type Entité 1 : {tup[0]['type']}\n
Entité 2 :\n
    Mention(s) Entité 2 : {tup[1]['mentions']}\n
    Type Entité 2 : {tup[1]['type']}\n
Indiquez la relation existante entre Entité 1 et Entité 2 selon les options fournies ou sélectionnez 'PAS_DE_RELATION' si aucune relation n'est détectée.""")
            inputs.extend([sent] * 1)
            outputs.extend([tup[-1]] * 1)
    
    return instructions, inputs, outputs


def get_finred_dataset(df, with_orig, with_cls):
    instructions, inputs, outputs = [], [], []
    tuples_list_fr = []
    
    sentences = df['text'].to_list()
    relations = df['relations_possibles'].to_list()
    #relations = list(set(df['relations'].to_list()))
    #print(relations)

    relations_fr = []
    for rel in relations:
        list_rel_fr = [relations_trad[r.strip().upper()] for r in rel.split(";")]
        relations_fr.append(" ; ".join(list_rel_fr))
    relations = relations_fr

    for i in range(len(df)):
        e1, e2, rel = df['entité1'][i], df['entité2'][i], df['relations'][i]
        rel_fr = relations_trad[rel]
        tuples_list_fr.append([e1, e2, rel_fr])
    
    for sent, tuples, rel_possibles in zip(sentences, tuples_list_fr, relations):
        rel_possibles = " ; ".join(list(set(rel_possibles.split(" ; "))))
        ins, i, o = get_instruction(sent, [tuples], rel_possibles, with_orig, with_cls)
        #ins, i, o = get_instruction(sent, [tuples], relations, with_orig, with_cls)
        
        instructions.extend(ins)
        inputs.extend(i)
        outputs.extend(o)
        
    print("longuer = ",len(outputs))
    print("exemple : ",outputs[:10])
    print("exemple Instruciton: ",instructions[10])  
    return Dataset.from_dict({
        'input': inputs,
        'output': outputs,
        'instruction': instructions
    })


In [46]:
test_dataset = get_finred_dataset(df_balanced_test, with_orig=False, with_cls=True)

longuer =  2392
exemple :  ['PAS_DE_RELATION', 'PAS_DE_RELATION', 'PAS_DE_RELATION', 'PAS_DE_RELATION', 'PAS_DE_RELATION', 'PAS_DE_RELATION', 'PAS_DE_RELATION', 'PAS_DE_RELATION', 'PAS_DE_RELATION', 'PAS_DE_RELATION']
exemple Instruciton:  D'après le contexte du texte d’entrée, identifiez la relation spécifique entre les deux entités suivantes. Choisissez une réponse parmi : [ ['DATE_DE_DÉBUT', 'CRÉÉ', 'EST_NÉ_A', 'RÉSIDE_DANS', 'EST_NÉ_LE', 'A_UNE_CATEGORIE', 'A_POUR_LONGUEUR', 'EST_EN_CONFLIT_AVEC', 'A_DES_CONSÉQUENCES', 'EST_MORT_LE', 'EST_EN_CONTACT_AVEC', 'A_ÉTÉ_CRÉÉ_EN', 'NOMBRE_DE_BLESSÉS', 'DATE_DE_FIN', 'INITIÉ', 'A_COULEUR', 'PAS_DE_RELATION', 'A_COMMENCÉ_EN', 'EST_DE_NATIONALITÉ', 'EST_ENREGISTRÉ_COMME', 'OPÈRE_EN', 'GENRE_FEMININ', 'EST_LOCALISÉ_EN', 'A_UNE_RELATION_FAMILIALE', 'NOMBRE_DE_MORTS', 'COOPÈRE_AVEC', 'A_POUR_HAUTEUR', 'A_UNE_CE_CONTROLE_SUR', 'A_UNE_QUANTITÉ', 'GENRE_MASCULIN', 'A_ÉTÉ_DISSOUTE_EN', 'EST_DE_TAILLE', 'MORT_EN', 'FAIT_PARTIE_DE'] ].

Entité 1 :

  

In [47]:
train_dataset = get_finred_dataset(df_balanced_train, with_orig=False, with_cls=True)

longuer =  5025
exemple :  ['PAS_DE_RELATION', 'PAS_DE_RELATION', 'PAS_DE_RELATION', 'PAS_DE_RELATION', 'PAS_DE_RELATION', 'PAS_DE_RELATION', 'PAS_DE_RELATION', 'PAS_DE_RELATION', 'PAS_DE_RELATION', 'PAS_DE_RELATION']
exemple Instruciton:  D'après le contexte du texte d’entrée, identifiez la relation spécifique entre les deux entités suivantes. Choisissez une réponse parmi : [ ['DATE_DE_DÉBUT', 'CRÉÉ', 'EST_NÉ_A', 'RÉSIDE_DANS', 'EST_NÉ_LE', 'A_UNE_CATEGORIE', 'A_POUR_LONGUEUR', 'EST_EN_CONFLIT_AVEC', 'A_DES_CONSÉQUENCES', 'EST_MORT_LE', 'EST_EN_CONTACT_AVEC', 'NOMBRE_DE_BLESSÉS', 'DATE_DE_FIN', 'INITIÉ', 'A_COULEUR', 'PAS_DE_RELATION', 'A_COMMENCÉ_EN', 'EST_DE_NATIONALITÉ', 'EST_ENREGISTRÉ_COMME', 'OPÈRE_EN', 'GENRE_FEMININ', 'EST_LOCALISÉ_EN', 'A_UNE_RELATION_FAMILIALE', 'NOMBRE_DE_MORTS', 'COOPÈRE_AVEC', 'A_POUR_HAUTEUR', 'A_UNE_CE_CONTROLE_SUR', 'A_UNE_QUANTITÉ', 'GENRE_MASCULIN', 'A_ÉTÉ_DISSOUTE_EN', 'EST_DE_TAILLE', 'MORT_EN', 'FAIT_PARTIE_DE'] ].

Entité 1 :

    Mention(s) Enti

#### data soumission 

In [54]:
test_dataset = get_finred_dataset(df_soumission5, with_orig=False, with_cls=True)

longuer =  73692
exemple :  ['PAS_DE_RELATION', 'PAS_DE_RELATION', 'PAS_DE_RELATION', 'PAS_DE_RELATION', 'PAS_DE_RELATION', 'PAS_DE_RELATION', 'PAS_DE_RELATION', 'PAS_DE_RELATION', 'PAS_DE_RELATION', 'PAS_DE_RELATION']
exemple Instruciton:  D'après le contexte du texte d’entrée, identifiez la relation spécifique entre les deux entités suivantes. Choisissez une réponse parmi : [ ['PAS_DE_RELATION'] ].

Entité 1 :

    Mention(s) Entité 1 : [{'value': 'FEAR', 'start': 261, 'end': 265}, {'value': 'FEAR', 'start': 1, 'end': 5}]

    Type Entité 1 : NON_GOVERNMENTAL_ORGANISATION

Entité 2 :

    Mention(s) Entité 2 : [{'value': 'appareil', 'start': 352, 'end': 360}, {'value': 'appareil', 'start': 594, 'end': 602}, {'value': 'avion commercial', 'start': 240, 'end': 256}]

    Type Entité 2 : MATERIEL

Indiquez la relation existante entre Entité 1 et Entité 2 selon les options fournies ou sélectionnez 'PAS_DE_RELATION' si aucune relation n'est détectée.


In [ ]:

finred_dataset = DatasetDict({
    'train': train_dataset,
    'test': test_dataset
})

finred_dataset.save_to_disk('fingpt-finred')
finred_dataset

Saving the dataset (0/1 shards):  92%|█████████▏| 68000/73692 [00:00<00:00, 138620.38 examples/s]